# 客户端服务器结构

客户端服务器结构是最常见的网络通讯结构,无论是http网站,网游,基本都是使用这个结构,它的特点是一个服务器主机可以同时和多个客户端主机交互,为客户端提供统一的功能.

TCP协议的一般分为长连接和短链接,一般来说短链接就是:

建立连接->请求->响应->关闭连接

而长连接就是:

建立连接-> 请求->响应->请求->响应....->关闭连接

比如http协议就是短连接协议,而像mongodb,mysql这样的数据库协议一般都提供长连接协议和短链接协议两种.

短链接比较好控制,但每次都要建立连接就比较消耗性能;而长连接不用频繁建立连接可以更加高效,但管理维护起来也更加麻烦.

## 短链接

我们定义一个这样的简单应用层协议:

1. 使用短链接,且由客户端主动断开连接
2. 传输的数据第一位为a,b,c,而后面则是一个数必须为一个数.
3. 响应为第一位d,后面为传入数的平方,立方,4次方,对应a,b,c3个第一位的请求

### 服务端协议实现

In [1]:
%%writefile short_server.py
import asyncio
class MyServerProtocol(asyncio.Protocol):
    HANDLERS = {
        "a":lambda x:x**2,
        "b":lambda x:x*x*x,
        "c":lambda x:x*x*x*x,
        "d":lambda x:x,
    }
    
    def connection_made(self, transport):
        peername = transport.get_extra_info('peername')
        print('Connection from {}'.format(peername))
        self.transport = transport
        
        
    def _decoder(self,data):
        message = data.decode()
        result = self.HANDLERS.get(message[0])(float(message[1:]))
        return result
    
    def _encoder(self,query,num):
        message = query+str(num)
        return message.encode()

    def data_received(self, data):
        result =self._decoder(data)
        self.transport.write(self._encoder("d",result))
        
    def connection_lost(self, exc):
        print('The client closed the connection')

if __name__=="__main__":
    loop = asyncio.get_event_loop()
    # Each client connection will create a new protocol instance
    coro = loop.create_server(MyServerProtocol, '127.0.0.1', 5000)
    server = loop.run_until_complete(coro)

    # Serve requests until Ctrl+C is pressed
    print('Serving on {}'.format(server.sockets[0].getsockname()))
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        pass

    # Close the server
    server.close()

Overwriting short_server.py


### 客户端协议实现

In [2]:
import asyncio
from functools import partial

class MyClientProtocol(asyncio.Protocol):
    HANDLERS = {
        "a":lambda x:x**2,
        "b":lambda x:x*x*x,
        "c":lambda x:x*x*x*x,
        "d":lambda x:x,
    }
    def __init__(self,loop=None):
        self.loop = loop or asyncio.get_event_loop()
        self.result = loop.create_future()
    
    def connection_made(self, transport):
        peername = transport.get_extra_info('peername')
        print('Connection to {}'.format(peername))
        self.transport = transport
        
        
    def _decoder(self,data):
        message = data.decode()
        result = self.HANDLERS.get(message[0])(float(message[1:]))
        return result
    
    def _encoder(self,query,num):
        message = query+str(num)
        return message.encode()

    def data_received(self, data):
        result =self._decoder(data)
        self.result.set_result(result)
        self.transport.close()
        
        

class Client:
    @staticmethod
    async def square(number,*,host= '127.0.0.1',port=5000,loop=None):
        
        prot_fac = partial(MyClientProtocol,loop=loop)
        tra, pro = await loop.create_connection(prot_fac,host=host,port=port)
        tra.write(pro._encoder("a",number))
        return await pro.result
        
        
    @staticmethod
    async def cube(number,*,host= '127.0.0.1',port=5000,loop=None):
        prot_fac = partial(MyClientProtocol,loop=loop)
        tra, pro = await loop.create_connection(prot_fac,host=host,port=port)
        tra.write(pro._encoder("b",number))
        return await pro.result
    
    @staticmethod
    async def four_square(number,*,host= '127.0.0.1',port=5000,loop=None):
        prot_fac = partial(MyClientProtocol,loop=loop)
        tra, pro = await loop.create_connection(prot_fac,host=host,port=port)
        tra.write(pro._encoder("c",number))
        return await pro.result
    
async def main(loop):
    print(await Client.square(2,loop=loop))
    print(await Client.cube(2,loop=loop))
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

Connection from ('127.0.0.1', 5000)
4.0
Connection from ('127.0.0.1', 5000)
8.0


上面的例子是最简单的一个原型实现,没有做任何异常处理的工作.

通常来说协议类比较适合写这种短链接的服务器和客户端,因为压根没用到tcp协议面向流的特性.

## 长连接

长连接和适合用于需要维护用户状态的服务,也适合那种同一客户端频繁访问的服务.通常网络游戏就都是使用的长连接.接下来我们试着写一个最简单的网络游戏---猜拳.

游戏规则:



### 客户端协议实现

In [ ]:
class LongClient:
     HANDLERS = {
        "a":lambda x:x**2,
        "b":lambda x:x*x*x,
        "c":lambda x:x*x*x*x,
        "d":lambda x:x,
    }
    
    def __init__(self,loop):
        self.loop=loop
        
    def _decoder(self,data):
        message = data.decode()
        result = self.HANDLERS.get(message[0])(message[1:])
        return result
    
    def _encoder(self,query,num):
        message = query+str(num)
        return message.encode()
        
    async def __aenter__(self):
        print('entering context')
        await self.connect()
        return self

    async def __aexit__(self, exc_type, exc, tb):
        print('exit context')
        self.close()
        
    async def connect(self):
        self.reader,self.writer = await asyncio.open_connection(host="0.0.0.0", port=5000,loop=self.loop)
        
    async def close(self):
        self.writer.close()
        
    async def square(self,number):
        self.writer.write(self._encoder("a",number))
        await self.writer.drain()